In [1]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [2]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\msflo\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [3]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets

In [5]:
df = pd.read_csv('data/audio_features_hot_100_1958_2019.csv')

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()
# Display dataframe
df.head()

,track_id,artist,track,spotify_genre,spotify_track_id,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,0.734,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,A Star Is Born Soundtrack,0.0,215733.0,88.0,0.572,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,Hollywood's Bleeding,1.0,196760.0,86.0,0.542,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,Bacc At It Again,1.0,228185.0,61.0,0.948,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0
5,The ArcherTaylor Swift,Taylor Swift,The Archer,"['dance pop', 'pop', 'post-teen pop']",3pHkh7d0lzM2AldUtz2x37,Lover,0.0,211240.0,76.0,0.292,...,0.0,-9.375,1.0,0.0401,0.12000,0.005690,0.0663,0.166,124.344,4.0


In [6]:
df['target'] = pd.qcut(df['spotify_track_popularity'],10, labels = False)

In [7]:
from sklearn.model_selection import train_test_split
y = pd.Series(df["target"])
X = df.drop(columns=["track_id", "artist", "track", "spotify_genre", "spotify_track_id", "spotify_track_album", "target", 'spotify_track_popularity' ])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
y.describe()

count    23559.000000
mean         4.435460
std          2.859693
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max          9.000000
Name: target, dtype: float64

In [9]:
X_train

,spotify_track_explicit,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
28040,0.0,203440.0,0.665,0.664,4.0,-9.480,1.0,0.0358,0.1860,0.000002,0.1020,0.960,88.679,4.0
17380,0.0,147653.0,0.434,0.780,0.0,-7.826,1.0,0.1630,0.6500,0.000000,0.0682,0.644,171.940,4.0
7310,0.0,251293.0,0.431,0.735,1.0,-6.995,1.0,0.0508,0.0945,0.000111,0.1170,0.360,139.984,4.0
5864,1.0,284600.0,0.736,0.549,8.0,-6.580,1.0,0.1290,0.0541,0.000000,0.1090,0.272,142.064,4.0
20132,0.0,227933.0,0.715,0.400,9.0,-13.250,1.0,0.0876,0.0760,0.000000,0.0516,0.706,96.607,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12776,0.0,204200.0,0.623,0.590,4.0,-5.014,1.0,0.0280,0.6460,0.000000,0.2950,0.579,137.071,4.0
20620,0.0,169533.0,0.628,0.693,2.0,-11.236,1.0,0.0301,0.0562,0.005020,0.1860,0.822,125.349,4.0
6085,0.0,263960.0,0.650,0.665,6.0,-12.763,0.0,0.0469,0.2080,0.000000,0.3040,0.725,135.598,4.0
14192,0.0,225560.0,0.802,0.738,0.0,-5.975,0.0,0.2360,0.0192,0.000000,0.2100,0.597,110.862,4.0


In [10]:
y_train

28040    0
17380    0
7310     6
5864     6
20132    1
        ..
12776    8
20620    0
6085     6
14192    5
252      9
Name: target, Length: 17669, dtype: int64

In [11]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [13]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [14]:
y_train_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [15]:
X_train_scaled.shape

(17669, 14)

In [20]:
model = Sequential()
model.add(Dense(units=10, activation='relu', input_dim=14))
model.add(Dense(units=10, activation='softmax'))

In [21]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                150       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 260
Trainable params: 260
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# set
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
553/553 - 0s - loss: 2.1004 - accuracy: 0.2055
Epoch 2/60
553/553 - 1s - loss: 2.1000 - accuracy: 0.2076
Epoch 3/60
553/553 - 1s - loss: 2.0991 - accuracy: 0.2057
Epoch 4/60
553/553 - 1s - loss: 2.0987 - accuracy: 0.2043
Epoch 5/60
553/553 - 1s - loss: 2.0984 - accuracy: 0.2084
Epoch 6/60
553/553 - 1s - loss: 2.0980 - accuracy: 0.2070
Epoch 7/60
553/553 - 1s - loss: 2.0974 - accuracy: 0.2060
Epoch 8/60
553/553 - 1s - loss: 2.0969 - accuracy: 0.2062
Epoch 9/60
553/553 - 1s - loss: 2.0968 - accuracy: 0.2062
Epoch 10/60
553/553 - 1s - loss: 2.0960 - accuracy: 0.2063
Epoch 11/60
553/553 - 1s - loss: 2.0960 - accuracy: 0.2075
Epoch 12/60
553/553 - 1s - loss: 2.0954 - accuracy: 0.2052
Epoch 13/60
553/553 - 1s - loss: 2.0951 - accuracy: 0.2057
Epoch 14/60
553/553 - 1s - loss: 2.0944 - accuracy: 0.2087
Epoch 15/60
553/553 - 1s - loss: 2.0939 - accuracy: 0.2066
Epoch 16/60
553/553 - 1s - loss: 2.0937 - accuracy: 0.2045
Epoch 17/60
553/553 - 2s - loss: 2.0932 - accuracy: 0.2071
Epoch 

In [24]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

185/185 - 0s - loss: 2.1080 - accuracy: 0.1902
Normal Neural Network - Loss: 2.1079962253570557, Accuracy: 0.19015279412269592


## Save the Model

In [ ]:
# save model 
# filename = 'Models/deep_audio.sav'
# joblib.dump(model, filename)

In [ ]:
model.save('Models/spotify_pop_DL.h5')